In [65]:
import statsmodels.stats.proportion
statsmodels.stats.proportion.proportions_ztest(526, 1000, value=0.5)

(1.6466121098225726, 0.099637799747829478)

In [39]:
def ztest_proportion(successes, trials, popmean=0.5, one_sided=False):
    se = popmean*(1-popmean)/trials
    p = successes/trials
    se = np.sqrt(se)
    z = (p-popmean)/se
    p = 1-stats.norm.cdf(abs(z))
    p *= 2-one_sided # if not one_sided: p *= 2
    return z, p

In [56]:
ztest_proportion(546, 1000, popmean=0.5)

(2.909295447354912, 0.0036224435050637993)

In [57]:
ztest_proportion(546, 1000, popmean=0.5, one_sided=True)

(2.909295447354912, 0.0018112217525318997)

In [58]:
statsmodels.stats.proportion.proportions_ztest(546, 1000, value=.5, alternative='smaller')

(2.9216862985268865, 0.99825929001428448)

In [59]:
ztest_proportion(546, 1000, popmean=0.5, one_sided=False)

(2.909295447354912, 0.0036224435050637993)

In [46]:
statsmodels.stats.proportion.proportions_ztest(546, 1000, value=.3, alternative='smaller', prop_var=.3)

(16.975612759821917, 1.0)

In [16]:
import scipy

In [18]:
scipy.stats.chisquare()

<function scipy.stats.contingency.chi2_contingency>

- https://www.r-bloggers.com/one-proportion-z-test-in-r/

old

In [71]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats
import sklearn.metrics
import math
import matplotlib.pyplot as plt


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages

# https://github.com/yang-zhang/ds-utils
import ds_utils.stats

In [2]:
base = rpackages.importr('base')

In [18]:
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages(ro.StrVector(['entropy', 'psych', 'vcd']))

/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: also installing the dependencies ‘zoo’, ‘mnormt’, ‘colorspace’, ‘lmtest’


  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://cloud.r-project.org/src/contrib/zoo_1.7-14.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Content type 'application/x-gzip'
  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning:  length 845540 bytes (825 KB)

  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: =
  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/opt/conda/lib/python3.6/site-pack

rpy2.rinterface.NULL

Or run a R kernal in Jupyter and run 
```
install.packages(c('entropy', 'psych', 'vcd'))
```

In [29]:
rpackages.importr('entropy')

rpy2.robjects.packages.Package as a <module 'entropy'>

This notebook demonstrates various methods to measure correlations between data (numerical and categorical), with code in Python and R, side by side. 

For exmaples, see https://github.com/yang-zhang/lang-python/blob/master/rpy2%20Tutorial.ipynb

# Make data

In [30]:
N = 500

## Numericals

In [31]:
some_numerical = np.random.uniform(0, 1, N)
some_numerical_with_noise = some_numerical + 0.1*np.random.randn(N)

some_numerical_r = ro.FloatVector(some_numerical)
some_numerical_with_noise_r = ro.FloatVector(some_numerical_with_noise)

## Categoricals

In [32]:
def generate_random_ints(num_categories, N):
    some_random_int = np.random.randint(0, num_categories, N)
    correlated_random_int = some_random_int.copy()
    
    for i in range(len(some_random_int)):
        if np.random.uniform(0, 1)>0.9:
            correlated_random_int[i]=np.random.randint(0, num_categories, 1)
    return some_random_int, correlated_random_int

In [33]:
some_random_int, correlated_random_int = generate_random_ints(3, N)

uncorrelated_random_int = generate_random_ints(3, N)[0]

some_categorical_r = ro.FactorVector(some_random_int)
correlated_categorical_r = ro.FactorVector(correlated_random_int)
uncorrelated_categorical_r = ro.FactorVector(uncorrelated_random_int)

## Numerical v.s. categorical

In [34]:
numerical_correlated_to_some_categorical = np.array([np.random.normal(c, 1) for c in some_random_int])
numerical_correlated_to_some_categorical_r = ro.FloatVector(numerical_correlated_to_some_categorical)

# Numerical v.s. numerical

## Pearson correlation coefficient

### Python

In [35]:
scipy.stats.pearsonr(some_numerical, some_numerical_with_noise)

(0.93948677679738879, 7.8494895038382033e-234)

### R

In [36]:
print(ro.r.cor(some_numerical_r, some_numerical_with_noise_r))

[1] 0.9394868



# Categorical v.s. categorical

## Chi-square

### Python

#### Correlated example

In [37]:
df = pd.DataFrame({
    '_': 0,
    'some_random_int': some_random_int,
    'correlated_random_int': correlated_random_int,
})

contingency_table = df.pivot_table(
    values='_',
    columns='some_random_int',
    index='correlated_random_int',
    aggfunc='count')

scipy.stats.chi2_contingency(contingency_table)

(785.04334960990047,
 1.3334331957856474e-168,
 4,
 array([[ 60.18 ,  62.9  ,  46.92 ],
        [ 62.658,  65.49 ,  48.852],
        [ 54.162,  56.61 ,  42.228]]))

#### Uncorrelated example

In [38]:
df = pd.DataFrame({
    '_': 0,
    'some_random_int': some_random_int,
    'correlated_random_int': uncorrelated_random_int,
})

contingency_table = df.pivot_table(
    values='_',
    columns='some_random_int',
    index='correlated_random_int',
    aggfunc='count')

scipy.stats.chi2_contingency(contingency_table)

(3.3288298033276136,
 0.50437743246185485,
 4,
 array([[ 64.428,  67.34 ,  50.232],
        [ 57.702,  60.31 ,  44.988],
        [ 54.87 ,  57.35 ,  42.78 ]]))

### R

#### Correlated example

In [42]:
res = ro.r('chisq.test')(some_categorical_r, correlated_categorical_r)
res.names.r_repr()
print(res[res.names.index('statistic')])
print(res[res.names.index('p.value')])

'c("statistic", "parameter", "p.value", "method", "data.name", \n"observed", "expected", "residuals", "stdres")'

X-squared 
 785.0433 

[1] 1.333433e-168



In [43]:
# An equivalent way
tb = ro.r('table')(some_categorical_r, correlated_categorical_r)
res = ro.r('chisq.test')(tb)
print(res[res.names.index('statistic')])
print(res[res.names.index('p.value')])

X-squared 
 785.0433 

[1] 1.333433e-168



In [44]:
np.array([[1,0],[0,3]])

array([[1, 0],
       [0, 3]])

In [45]:
scipy.stats.chi2_contingency(np.array([[1,0],[0,3]]))

(0.44444444444444442, 0.50498507509384571, 1, array([[ 0.25,  0.75],
        [ 0.75,  2.25]]))

#### Uncorrelated example

In [46]:
res = ro.r('chisq.test')(some_categorical_r, uncorrelated_categorical_r)
res.names.r_repr()
print(res[res.names.index('statistic')])
print(res[res.names.index('p.value')])

'c("statistic", "parameter", "p.value", "method", "data.name", \n"observed", "expected", "residuals", "stdres")'

X-squared 
  3.32883 

[1] 0.5043774



## Mutual information

### Python

In [47]:
value_counts_some_random_int = np.bincount(some_random_int)
H_some_random_int = scipy.stats.entropy(value_counts_some_random_int)

In [48]:
def calculate_joint_entropy(x, y):
    df = pd.DataFrame(np.stack((x, y), axis=1))
    df.columns = ['a', 'b']
    df_value_counts_joined = df.groupby(['a', 'b']).size().reset_index().rename(columns={0:'count'})
    value_counts_joined = df_value_counts_joined['count']
    return scipy.stats.entropy(value_counts_joined)

#### Correlated example

In [49]:
value_counts_correlated_random_int = np.bincount(correlated_random_int)
H_correlated_random_int = scipy.stats.entropy(value_counts_correlated_random_int)

H_some_random_int_joint_correlated_random_int = calculate_joint_entropy(some_random_int, correlated_random_int)

mutual_some_random_int_between_correlated_random_int = H_some_random_int + \
H_correlated_random_int - H_some_random_int_joint_correlated_random_int

mutual_some_random_int_between_correlated_random_int_another_method = sklearn.metrics.mutual_info_score(some_random_int, correlated_random_int)

In [50]:
H_some_random_int
H_correlated_random_int
mutual_some_random_int_between_correlated_random_int
mutual_some_random_int_between_correlated_random_int_another_method

1.0907974207086875

1.096765620547882

0.782311594319689

0.78231159431968833

#### Uncorrelated example

In [51]:
value_counts_uncorrelated_random_int = np.bincount(uncorrelated_random_int)
H_uncorrelated_random_int = scipy.stats.entropy(value_counts_uncorrelated_random_int)
H_some_random_int_joint_uncorrelated_random_int = calculate_joint_entropy(some_random_int, uncorrelated_random_int)

mutual_some_random_int_between_uncorrelated_random_int = H_some_random_int + \
H_uncorrelated_random_int - H_some_random_int_joint_uncorrelated_random_int

mutual_some_random_int_between_uncorrelated_random_int_another_method = sklearn.metrics.mutual_info_score(some_random_int, uncorrelated_random_int)

In [52]:
H_some_random_int
H_uncorrelated_random_int
mutual_some_random_int_between_uncorrelated_random_int
mutual_some_random_int_between_uncorrelated_random_int_another_method

1.0907974207086875

1.0963253126182673

0.0033193465887326745

0.0033193465887322859

### R

In [53]:
H_some_categorical_r = ro.r('entropy')(ro.r('table')(some_categorical_r))[0]

##### Correlated example

In [54]:
H_correlated_categorical_r = ro.r('entropy')(ro.r('table')(correlated_categorical_r))[0]
tb_cross = ro.r('table')(some_categorical_r, correlated_categorical_r)
H_some_categorical_r_joint_correlated_categorical_r = ro.r('entropy')(tb_cross)[0]
mutual_some_categorical_r_between_correlated_categorical_r = H_some_categorical_r + H_correlated_categorical_r - H_some_categorical_r_joint_correlated_categorical_r
mutual_some_categorical_r_between_correlated_categorical_r_another_method = ro.r('mi.empirical')(tb_cross)[0]

In [55]:
H_some_categorical_r
H_correlated_categorical_r
mutual_some_categorical_r_between_correlated_categorical_r
mutual_some_categorical_r_between_correlated_categorical_r_another_method

1.0907974207086875

1.096765620547882

0.782311594319689

0.7823115943196887

In [56]:
H_some_categorical_r_joint_correlated_categorical_r

1.4052514469368806

##### Uncorrelated example

In [57]:
H_uncorrelated_categorical_r = ro.r('entropy')(ro.r('table')(uncorrelated_categorical_r))[0]
tb_cross = ro.r('table')(some_categorical_r, uncorrelated_categorical_r)
H_some_categorical_r_joint_uncorrelated_categorical_r = ro.r('entropy')(tb_cross)[0]
mutual_some_categorical_r_between_uncorrelated_categorical_r = H_some_categorical_r + H_uncorrelated_categorical_r - H_some_categorical_r_joint_uncorrelated_categorical_r
mutual_some_categorical_r_between_uncorrelated_categorical_r_another_method = ro.r('mi.empirical')(tb_cross)[0]

In [58]:
H_some_categorical_r
H_uncorrelated_categorical_r
mutual_some_categorical_r_between_uncorrelated_categorical_r
mutual_some_categorical_r_between_uncorrelated_categorical_r_another_method

1.0907974207086875

1.0963253126182673

0.0033193465887326745

0.0033193465887325995

# Numerical v.s. categorical

## t-test

### Python

#### Correlated example

In [59]:
numerical_correlated_to_some_categorical_0 = numerical_correlated_to_some_categorical[np.where(some_random_int==0)]
numerical_correlated_to_some_categorical_1 = numerical_correlated_to_some_categorical[np.where(some_random_int==1)]
scipy.stats.ttest_ind(numerical_correlated_to_some_categorical_0, 
                      numerical_correlated_to_some_categorical_1, 
                      axis=0, equal_var=True)

Ttest_indResult(statistic=-8.8757458951155712, pvalue=3.3213069397465049e-17)

#### Uncorrelated example

In [60]:
some_numerical_with_noise_0=some_numerical_with_noise[np.where(some_random_int==0)]
some_numerical_with_noise_1=some_numerical_with_noise[np.where(some_random_int==1)]
scipy.stats.ttest_ind(some_numerical_with_noise_0, 
                      some_numerical_with_noise_1,
                      axis=0, equal_var=True)

Ttest_indResult(statistic=-3.1502702601174768, pvalue=0.0017677398984381884)

### R

#### Correlated example

In [62]:
numerical_correlated_to_some_categorical_0_r = ro.FloatVector(numerical_correlated_to_some_categorical_0)
numerical_correlated_to_some_categorical_1_r = ro.FloatVector(numerical_correlated_to_some_categorical_1)
res = ro.r('t.test')(numerical_correlated_to_some_categorical_0_r, 
                     numerical_correlated_to_some_categorical_1_r)
res.names.r_repr()
print(res[res.names.index('statistic')])
print(res[res.names.index('p.value')])

'c("statistic", "parameter", "p.value", "conf.int", "estimate", \n"null.value", "alternative", "method", "data.name")'

        t 
-8.884021 

[1] 3.124753e-17



#### Uncorrelated example

In [63]:
some_numerical_with_noise_0_r = ro.FloatVector(some_numerical_with_noise_0)
some_numerical_with_noise_1_r = ro.FloatVector(some_numerical_with_noise_1)
res = ro.r('t.test')(some_numerical_with_noise_0_r, 
                     some_numerical_with_noise_1_r)
res.names.r_repr()
print(res[res.names.index('statistic')])
print(res[res.names.index('p.value')])

'c("statistic", "parameter", "p.value", "conf.int", "estimate", \n"null.value", "alternative", "method", "data.name")'

        t 
-3.151858 

[1] 0.001758482



## Analysis of Variance

### Python

#### Correlated example

In [64]:
numerical_correlated_to_some_categorical_values = []
for v in set(some_random_int):
    numerical_correlated_to_some_categorical_values.append(
        numerical_correlated_to_some_categorical[np.where(some_random_int==v)]
    )

In [65]:
scipy.stats.f_oneway(*numerical_correlated_to_some_categorical_values)

F_onewayResult(statistic=164.10411489014581, pvalue=1.8989795743851969e-55)

In [66]:
df = pd.DataFrame({
        'col_cat': some_random_int,
        'col_num': numerical_correlated_to_some_categorical,
    })

In [72]:
ds_utils.stats.df_anova(df, 'col_num', 'col_cat')

F_onewayResult(statistic=164.10411489014575, pvalue=1.8989795743851969e-55)

#### Uncorrelated example

In [73]:
some_numerical_with_noise_values = []
for v in set(some_random_int):
    some_numerical_with_noise_values.append(
        some_numerical_with_noise[np.where(some_random_int==v)]
    )

In [74]:
scipy.stats.f_oneway(*some_numerical_with_noise_values)

F_onewayResult(statistic=4.8802315262605536, pvalue=0.007963163070415346)

### R

#### Correlated example

In [75]:
df = ro.DataFrame({'num':numerical_correlated_to_some_categorical_r, 'cat': some_categorical_r})
aov_res = ro.r('aov')(ro.Formula('num~cat'), df)
print(base.summary(aov_res))

             Df Sum Sq Mean Sq F value Pr(>F)    
cat           2  334.2  167.12   164.1 <2e-16 ***
Residuals   497  506.1    1.02                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



#### Uncorrelated example

In [76]:
df = ro.DataFrame({'num':some_numerical_with_noise_r, 'cat': some_categorical_r})
aov_res = ro.r('aov')(ro.Formula('num~cat'), df)
print(base.summary(aov_res))

             Df Sum Sq Mean Sq F value  Pr(>F)   
cat           2   0.89  0.4428    4.88 0.00796 **
Residuals   497  45.09  0.0907                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



References: 
- http://stats.stackexchange.com/questions/108007/correlations-with-categorical-variables
- http://www.ats.ucla.edu/stat/mult_pkg/whatstat/default.htm